# fortune-500-employment-validation
SanjayKAroraPhD@gmail.com<br>
March 2020<br>

## Description
* Pull in employment data collected via the Forbes website for 2019 Fortune 500 firms and Global 500 firms
* Import employment data from EAGER study
* Clean both datasets, fuzzy match on firm name, then export as xlsx for manual inspection / fixing
* Import cleaned data
* Join imported cleaned data with keys to match on Forbes employment data

In [1]:
import csv
import pandas as pd
import numpy as np
import difflib
import re
import matplotlib.pyplot as plt

In [24]:
# specify input/output files
BASE_DIR = '/Users/sanjay/dev/EAGER'
f_in = BASE_DIR + '/code/analysis/fortune500/fortune-global-500-2019-emps.txt' # forbes data
e_in = BASE_DIR + '/data/orgs/emps/all_firms_with_emps_NO_FORBES.csv' # eager (employment) data 
x_out = BASE_DIR + '/code/analysis/fortune500/manual_inspection_needed.xlsx' # output file for manual inspection
x_in = BASE_DIR + '/code/analysis/fortune500/manual_inspection_done_NO_FORBES.xlsx'

f_out = BASE_DIR + '/code/analysis/fortune500/to_visualize_in_R.csv'

In [25]:
# import data
with open(f_in, 'r') as f:
    reader_1 = csv.reader(f, delimiter='|')
    fortune500 = list(reader_1)
    
e_df = pd.read_csv(e_in)
e_df.head()

,firm_name,url,li_emps,median_gw_emps,emp_diff,max_emps,size_state,state_diff,base_domain,subsidiary_adjust
0,Two Blades Foundation,2blades.org/,6.0,NaN,NaN,6.0,FirmSize.MICRO_FIRM,1,2blades.org,0
1,3M Innovative Properties Company,3m.com/,60593.0,NaN,NaN,60593.0,FirmSize.LARGE_FIRM,5,3m.com,0
2,Advanced Aqua Group,aadvancedaqua.com/,3.0,NaN,NaN,3.0,FirmSize.MICRO_FIRM,1,aadvancedaqua.com,0
3,ABB AB,abb.com/,100487.0,NaN,NaN,100487.0,FirmSize.LARGE_FIRM,5,abb.com,0
4,AbbVie Inc.,abbvie.com/,26260.0,29000.0,2740.0,29000.0,FirmSize.LARGE_FIRM,0,abbvie.com,0


In [26]:
f500_df = pd.DataFrame (fortune500, columns=['data'])
f500_df.head()

,data
0,1
1,Walmart
2,"2,200,000"
3,2
4,Exxon Mobil


In [27]:
# fix fortune 500 copied data
firm_df = f500_df.groupby(np.repeat(np.arange(len(f500_df) / 3), 3))['data'].apply(lambda x: x.values.reshape(1,3)[0])
firm_df.head()

0.0             [1, Walmart, 2,200,000]
1.0            [2, Exxon Mobil, 71,000]
2.0                 [3, Apple, 132,000]
3.0    [4, Berkshire Hathaway, 389,000]
4.0            [5, Amazon.com, 647,500]
Name: data, dtype: object

In [28]:
# change column headers
companies = firm_df.apply(pd.Series)
g = companies.rename(columns={0: "rank", 1: "name", 2: "emps"})
print (g.shape)
g.shape
g.head()

(1000, 3)


,rank,name,emps
0.0,1,Walmart,"2,200,000"
1.0,2,Exxon Mobil,"71,000"
2.0,3,Apple,"132,000"
3.0,4,Berkshire Hathaway,"389,000"
4.0,5,Amazon.com,"647,500"


In [29]:
# clean firm function
def clean_firm_name (firm):
    firm_clnd = re.sub('(\.com|\.|,|&| group| technologies| corporation| incorporated| llc| inc| international| gmbh| ltd)', '', firm, flags=re.IGNORECASE).rstrip()
    return firm_clnd

g['clnd_name'] = g['name'].apply (lambda x: clean_firm_name (clean_firm_name(x)))
e_df['clnd_name'] = e_df['firm_name'].apply (lambda x: clean_firm_name (clean_firm_name(x)))

print (g.shape)
display(g.head())

print (e_df.shape)
display(e_df.head())

(1000, 4)


,rank,name,emps,clnd_name
0.0,1,Walmart,"2,200,000",Walmart
1.0,2,Exxon Mobil,"71,000",Exxon Mobil
2.0,3,Apple,"132,000",Apple
3.0,4,Berkshire Hathaway,"389,000",Berkshire Hathaway
4.0,5,Amazon.com,"647,500",Amazon


(1492, 11)


,firm_name,url,li_emps,median_gw_emps,emp_diff,max_emps,size_state,state_diff,base_domain,subsidiary_adjust,clnd_name
0,Two Blades Foundation,2blades.org/,6.0,NaN,NaN,6.0,FirmSize.MICRO_FIRM,1,2blades.org,0,Two Blades Foundation
1,3M Innovative Properties Company,3m.com/,60593.0,NaN,NaN,60593.0,FirmSize.LARGE_FIRM,5,3m.com,0,3M Innovative Properties Company
2,Advanced Aqua Group,aadvancedaqua.com/,3.0,NaN,NaN,3.0,FirmSize.MICRO_FIRM,1,aadvancedaqua.com,0,Advanced Aqua
3,ABB AB,abb.com/,100487.0,NaN,NaN,100487.0,FirmSize.LARGE_FIRM,5,abb.com,0,ABB AB
4,AbbVie Inc.,abbvie.com/,26260.0,29000.0,2740.0,29000.0,FirmSize.LARGE_FIRM,0,abbvie.com,0,AbbVie


In [30]:
# fuzzy match function
def match (x): 
    return difflib.get_close_matches(x, g['clnd_name'], n=1, cutoff=0.7)

e_df['fg500_match'] = e_df['clnd_name'].apply(lambda x: match(x))

In [31]:
# clean fuzzy matches
temp = e_df['fg500_match']
zemp = []
index = 0
for t in temp: 
    if t == []:
        zemp.append('') 
    else:
        zemp.append(t[0])
    index += 1

In [32]:
# attached cleaned fuzzy match data and print out for manual inspection 
e_df['fg500_match'] = zemp
display (e_df)
e_df.to_excel (x_out)

,firm_name,url,li_emps,median_gw_emps,emp_diff,max_emps,size_state,state_diff,base_domain,subsidiary_adjust,clnd_name,fg500_match
0,Two Blades Foundation,2blades.org/,6.0,NaN,NaN,6.0,FirmSize.MICRO_FIRM,1,2blades.org,0,Two Blades Foundation,
1,3M Innovative Properties Company,3m.com/,60593.0,NaN,NaN,60593.0,FirmSize.LARGE_FIRM,5,3m.com,0,3M Innovative Properties Company,
2,Advanced Aqua Group,aadvancedaqua.com/,3.0,NaN,NaN,3.0,FirmSize.MICRO_FIRM,1,aadvancedaqua.com,0,Advanced Aqua,Advance Auto Parts
3,ABB AB,abb.com/,100487.0,NaN,NaN,100487.0,FirmSize.LARGE_FIRM,5,abb.com,0,ABB AB,
4,AbbVie Inc.,abbvie.com/,26260.0,29000.0,2740.0,29000.0,FirmSize.LARGE_FIRM,0,abbvie.com,0,AbbVie,AbbVie
5,Google Inc.,abc.xyz/,8.0,85050.0,85042.0,85050.0,FirmSize.LARGE_FIRM,4,abc.xyz,0,Google,
6,Abengoa Bioenergy New Technologies,abengoa.com/,5799.0,154.0,5645.0,5799.0,FirmSize.LARGE_FIRM,2,abengoa.com,0,Abengoa Bioenergy New,
7,Ablexis,ablexis.com/,2.0,NaN,NaN,2.0,FirmSize.MICRO_FIRM,1,ablexis.com,0,Ablexis,
8,SII Semiconductor Corporation,ablic.com/en/semicon/,3.0,NaN,NaN,3.0,FirmSize.MICRO_FIRM,1,ablic.com,0,SII Semiconductor,ON Semiconductor
9,ACACIA RESEARCH GROUP LLC,acaciaresearch.com/,2.0,13.0,11.0,13.0,FirmSize.SMALL_FIRM,1,acaciaresearch.com,0,ACACIA RESEARCH,


ModuleNotFoundError: No module named 'openpyxl'

In [33]:
# read in for manual inspection 
xin_df = pd.read_excel (x_in)
x_df = xin_df[xin_df['fg500_match'].notnull()]
print (x_df.shape)
x_df.head()
x_df['clnd_fg500'] = x_df['fg500_match'].apply (lambda x: clean_firm_name (clean_firm_name(x)))
x_df.head()

(145, 13)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Unnamed: 0,firm_name,url,li_emps,median_gw_emps,emp_diff,max_emps,size_state,state_diff,base_domain,subsidiary_adjust,clnd_name,fg500_match,clnd_fg500
0,419,Hon Hai Precision Industry Co.,foxconn.com/,224.0,503500.0,503276.0,7000.0,FirmSize.LARGE_FIRM,2,foxconn.com,0,Hon Hai Precision Industry Co,Hon Hai Precision Industry,Hon Hai Precision Industry
1,1135,Tata Consultancy Services Limited,tcs.com/,380007.0,394998.0,14991.0,394998.0,FirmSize.LARGE_FIRM,0,tcs.com,0,Tata Consultancy Services Limited,Tata Motors,Tata Motors
2,504,Hitachi,hitachi.com/,10457.0,319565.5,309108.5,303887.0,FirmSize.LARGE_FIRM,0,hitachi.com,0,Hitachi,Hitachi,Hitachi
3,439,GENERAL ELECTRIC COMPANY,ge.com/,2534.0,313000.0,310466.0,2534.0,FirmSize.LARGE_FIRM,0,ge.com,0,GENERAL ELECTRIC COMPANY,General Electric,General Electric
4,976,Samsung Electronics,samsung.com/us/,85449.0,171578.0,86129.0,85449.0,FirmSize.LARGE_FIRM,0,samsung.com,1,Samsung Electronics,Samsung Electronics,Samsung Electronics


In [34]:
# drop duplicates across the fortune 500 and global 500 firms
g.drop_duplicates(subset ="clnd_name", inplace = True) 
print (g.shape)

(891, 4)


In [35]:
# merge left, firm-assignees to global-fortune firms
y_df = x_df.merge(g, left_on='clnd_fg500', right_on='clnd_name', how='left')
print(y_df.shape)
y_df.head()

(145, 18)


,Unnamed: 0,firm_name,url,li_emps,median_gw_emps,emp_diff,max_emps,size_state,state_diff,base_domain,subsidiary_adjust,clnd_name_x,fg500_match,clnd_fg500,rank,name,emps,clnd_name_y
0,419,Hon Hai Precision Industry Co.,foxconn.com/,224.0,503500.0,503276.0,7000.0,FirmSize.LARGE_FIRM,2,foxconn.com,0,Hon Hai Precision Industry Co,Hon Hai Precision Industry,Hon Hai Precision Industry,23,Hon Hai Precision Industry,"667,680",Hon Hai Precision Industry
1,1135,Tata Consultancy Services Limited,tcs.com/,380007.0,394998.0,14991.0,394998.0,FirmSize.LARGE_FIRM,0,tcs.com,0,Tata Consultancy Services Limited,Tata Motors,Tata Motors,265,Tata Motors,"81,090",Tata Motors
2,504,Hitachi,hitachi.com/,10457.0,319565.5,309108.5,303887.0,FirmSize.LARGE_FIRM,0,hitachi.com,0,Hitachi,Hitachi,Hitachi,102,Hitachi,"295,941",Hitachi
3,439,GENERAL ELECTRIC COMPANY,ge.com/,2534.0,313000.0,310466.0,2534.0,FirmSize.LARGE_FIRM,0,ge.com,0,GENERAL ELECTRIC COMPANY,General Electric,General Electric,21,General Electric,"283,000",General Electric
4,976,Samsung Electronics,samsung.com/us/,85449.0,171578.0,86129.0,85449.0,FirmSize.LARGE_FIRM,0,samsung.com,1,Samsung Electronics,Samsung Electronics,Samsung Electronics,15,Samsung Electronics,"309,630",Samsung Electronics


In [36]:
# subset based on those firms not missing values
subset_df = y_df[y_df['clnd_fg500'].notnull()]
subset_df.drop_duplicates(subset ="clnd_fg500", inplace = True) 
print(subset_df.shape)
subset_df.to_csv(f_out)

(111, 18)
